# Uncertainty Forest (UF) Demo
This demo provides the basic use cases of the `uf` algorithm.

In [1]:
import numpy as np
import time

from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import entropy

from joblib import Parallel, delayed

In [6]:
def uf(X, y, n_estimators = 300, max_samples = .4, base = np.exp(1), kappa = 3):
    
    # Build forest with default parameters.
    model = BaggingClassifier(DecisionTreeClassifier(), 
                              n_estimators=n_estimators, 
                              max_samples=max_samples, 
                              bootstrap=False)
    model.fit(X, y)
    n = X.shape[0]
    K = model.n_classes_
    _, y = np.unique(y, return_inverse=True)
    
    cond_entropy = 0
    for tree_idx, tree in enumerate(model):
        # Find the indices of the training set used for partition.
        sampled_indices = model.estimators_samples_[tree_idx]
        unsampled_indices = np.delete(np.arange(0,n), sampled_indices)
        
        # Randomly split the rest into voting and evaluation.
        total_unsampled = len(unsampled_indices)
        np.random.shuffle(unsampled_indices)
        vote_indices = unsampled_indices[:total_unsampled//2]
        eval_indices = unsampled_indices[total_unsampled//2:]
        
        # Store the posterior in a num_nodes-by-num_classes matrix.
        # Posteriors in non-leaf cells will be zero everywhere
        # and later changed to uniform.
        node_counts = tree.tree_.n_node_samples
        class_counts = np.zeros((len(node_counts), K))
        est_nodes = tree.apply(X[vote_indices])
        est_classes = y[vote_indices]
        for i in range(len(est_nodes)):
            class_counts[est_nodes[i], est_classes[i]] += 1
        
        row_sums = class_counts.sum(axis=1) # Total number of estimation points in each leaf.
        row_sums[row_sums == 0] = 1 # Avoid divide by zero.
        class_probs = class_counts / row_sums[:, None]
        
        # Make the nodes that have no estimation indices uniform.
        # This includes non-leaf nodes, but that will not affect the estimate.
        class_probs[np.argwhere(class_probs.sum(axis = 1) == 0)] = [1 / K]*K
        
        # Apply finite sample correction and renormalize.
        where_0 = np.argwhere(class_probs == 0)
        for elem in where_0:
            class_probs[elem[0], elem[1]] = 1 / (kappa*class_counts.sum(axis = 1)[elem[0]])
        row_sums = class_probs.sum(axis=1)
        class_probs = class_probs / row_sums[:, None]
        
        # Place evaluation points in their corresponding leaf node.
        # Store evaluation posterior in a num_eval-by-num_class matrix.
        eval_class_probs = class_probs[tree.apply(X[eval_indices])]
        # eval_class_probs = [class_probs[x] for x in tree.apply(X[eval_indices])]
        eval_entropies = [entropy(posterior) for posterior in eval_class_probs]
        cond_entropy += np.mean(eval_entropies)
      
    return cond_entropy / n_estimators

## Estimate the Conditional Entropy of `Y` given `X`
Use `X_train` and `y_train` to estimate the posterior and conditional entropy.

In [7]:
n_class = 1000
d = 10
classes = [-1, 0, 1]
n = n_class*len(classes)

X = np.array(np.concatenate([np.random.multivariate_normal(k*np.ones(d), np.eye(d), n_class) for k in classes]))
y = np.array(np.concatenate([k*np.ones(n_class) for k in classes]))
print(X.shape)
print(y.shape)

(3000, 10)
(3000,)


In [8]:
n_estimators = 300

start_time = time.time()
cond_entropy = uf(X, y, n_estimators = n_estimators)
elapsed_time = time.time() - start_time
print("Elapsed time with n =", n, "and", n_estimators, "trees.")
print(elapsed_time)

print("H(Y|X) =", cond_entropy)

Elapsed time with n = 3000 and 300 trees.
25.883588075637817
H(Y|X) = 0.5563671200406107


## Estimate the mutual information
Use the MLE for entropy of `y`, and subtract the conditional entropy for the mutual information estimate.

In [5]:
# Compute frequency distribution for y.
_, counts = np.unique(y, return_counts=True)
entropy_y = entropy(counts, base=np.exp(1))

mutual_info = entropy_y - cond_entropy
print("I(X; Y) = %f" % mutual_info)

I(X; Y) = 0.539756
